In [ ]:
# PLANTILLA LANGCHAIN - SISTEMA MULTIAGENTE
# =========================================

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.tools import Tool, DuckDuckGoSearchRun
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
import asyncio
from typing import Dict, List, Any
import os

# PASO 1: Configuración inicial
# =============================
os.environ["OPENAI_API_KEY"] = "tu-api-key-aquí"

# Configurar el modelo LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7,
    max_tokens=2000
)

# PASO 2: Definición de herramientas
# ==================================

# Herramienta de búsqueda
search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="Buscador_Web",
    description="Útil para buscar información actual en internet sobre cualquier tema",
    func=search.run
)

# Herramienta personalizada para análisis
def analizar_texto(texto: str) -> str:
    """Analiza texto y extrae puntos clave"""
    # Aquí podrías implementar lógica más compleja
    lineas = texto.split('\n')
    puntos_clave = [linea.strip() for linea in lineas if linea.strip() and len(linea) > 20]
    return f"Puntos clave encontrados:\n" + "\n".join(f"- {punto}" for punto in puntos_clave[:5])

analisis_tool = Tool(
    name="Analizador_Texto",
    description="Analiza texto y extrae los puntos más importantes",
    func=analizar_texto
)

# PASO 3: Clase base para agentes
# ===============================

class AgenteBase:
    """Clase base para todos los agentes del sistema"""
    
    def __init__(self, nombre: str, rol: str, herramientas: List[Tool], sistema_prompt: str):
        self.nombre = nombre
        self.rol = rol
        self.herramientas = herramientas
        self.llm = llm
        self.memoria = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # Crear el prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", sistema_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        # Crear el agente
        self.agente = create_openai_functions_agent(
            llm=self.llm,
            tools=self.herramientas,
            prompt=self.prompt
        )
        
        # Crear el executor
        self.executor = AgentExecutor(
            agent=self.agente,
            tools=self.herramientas,
            memory=self.memoria,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=3
        )
    
    def ejecutar(self, tarea: str) -> str:
        """Ejecuta una tarea específica"""
        try:
            resultado = self.executor.invoke({"input": tarea})
            return resultado['output']
        except Exception as e:
            return f"Error en {self.nombre}: {str(e)}"
    
    def obtener_historial(self) -> List[Any]:
        """Obtiene el historial de conversación"""
        return self.memoria.chat_memory.messages

# PASO 4: Definición de agentes especializados
# ============================================

class AgenteInvestigador(AgenteBase):
    """Agente especializado en investigación"""
    
    def __init__(self):
        sistema_prompt = """
        Eres un investigador senior especializado en encontrar información precisa y relevante.
        
        Tu trabajo es:
        1. Buscar información actualizada sobre el tema solicitado
        2. Verificar la confiabilidad de las fuentes
        3. Estructurar la información de manera clara
        4. Proporcionar un resumen objetivo con fuentes citadas
        
        Siempre menciona las fuentes de tu información y sé crítico con la calidad de los datos.
        """
        
        super().__init__(
            nombre="Investigador",
            rol="Investigador Senior",
            herramientas=[search_tool],
            sistema_prompt=sistema_prompt
        )

class AgenteAnalista(AgenteBase):
    """Agente especializado en análisis de datos"""
    
    def __init__(self):
        sistema_prompt = """
        Eres un analista de datos experto en procesar información y extraer insights valiosos.
        
        Tu trabajo es:
        1. Analizar la información proporcionada
        2. Identificar patrones y tendencias importantes
        3. Extraer conclusiones basadas en evidencia
        4. Proporcionar recomendaciones estructuradas
        
        Mantén un enfoque objetivo y basado en datos en todos tus análisis.
        """
        
        super().__init__(
            nombre="Analista",
            rol="Analista de Datos",
            herramientas=[analisis_tool],
            sistema_prompt=sistema_prompt
        )

class AgenteRedactor(AgenteBase):
    """Agente especializado en redacción y documentación"""
    
    def __init__(self):
        sistema_prompt = """
        Eres un redactor técnico experto en crear documentos claros y bien estructurados.
        
        Tu trabajo es:
        1. Organizar información compleja de manera coherente
        2. Escribir contenido claro y accesible
        3. Estructurar documentos con formato profesional
        4. Adaptar el lenguaje según la audiencia objetivo
        
        Siempre prioriza la claridad y la estructura lógica en tus documentos.
        """
        
        super().__init__(
            nombre="Redactor",
            rol="Redactor Técnico",
            herramientas=[],
            sistema_prompt=sistema_prompt
        )

# PASO 5: Coordinador de agentes
# ==============================

class CoordinadorMultiagente:
    """Coordina la comunicación y flujo de trabajo entre agentes"""
    
    def __init__(self):
        self.investigador = AgenteInvestigador()
        self.analista = AgenteAnalista()
        self.redactor = AgenteRedactor()
        self.resultados = {}
    
    def ejecutar_flujo_completo(self, tema: str) -> Dict[str, str]:
        """
        Ejecuta el flujo completo de investigación, análisis y redacción
        
        Args:
            tema (str): Tema a investigar
            
        Returns:
            Dict[str, str]: Resultados de cada fase
        """
        print(f"🚀 Iniciando flujo multiagente para: {tema}")
        
        # FASE 1: Investigación
        print("\n📊 FASE 1: Investigación")
        tarea_investigacion = f"""
        Investiga a fondo sobre {tema}. Incluye:
        - Definición y conceptos clave
        - Estado actual del tema
        - Principales aplicaciones o casos de uso
        - Tendencias y desarrollos recientes
        - Fuentes confiables
        
        Proporciona un informe estructurado con toda la información encontrada.
        """
        
        resultado_investigacion = self.investigador.ejecutar(tarea_investigacion)
        self.resultados['investigacion'] = resultado_investigacion
        
        # FASE 2: Análisis
        print("\n🔍 FASE 2: Análisis")
        tarea_analisis = f"""
        Analiza la siguiente información sobre {tema}:
        
        {resultado_investigacion}
        
        Proporciona:
        - Análisis de los puntos más importantes
        - Identificación de tendencias clave
        - Evaluación de oportunidades y desafíos
        - Conclusiones basadas en evidencia
        - Recomendaciones estructuradas
        """
        
        resultado_analisis = self.analista.ejecutar(tarea_analisis)
        self.resultados['analisis'] = resultado_analisis
        
        # FASE 3: Redacción
        print("\n✍️ FASE 3: Redacción Final")
        tarea_redaccion = f"""
        Crea un documento final profesional sobre {tema} usando:
        
        INVESTIGACIÓN:
        {resultado_investigacion}
        
        ANÁLISIS:
        {resultado_analisis}
        
        El documento debe tener:
        - Título y introducción
        - Secciones bien estructuradas
        - Conclusiones claras
        - Formato markdown
        - Lenguaje profesional pero accesible
        """
        
        resultado_final = self.redactor.ejecutar(tarea_redaccion)
        self.resultados['documento_final'] = resultado_final
        
        return self.resultados
    
    def ejecutar_tarea_personalizada(self, agente_nombre: str, tarea: str) -> str:
        """
        Ejecuta una tarea específica en un agente particular
        
        Args:
            agente_nombre (str): 'investigador', 'analista', o 'redactor'
            tarea (str): Descripción de la tarea
            
        Returns:
            str: Resultado de la tarea
        """
        agentes = {
            'investigador': self.investigador,
            'analista': self.analista,
            'redactor': self.redactor
        }
        
        if agente_nombre not in agentes:
            return f"Agente '{agente_nombre}' no encontrado"
        
        return agentes[agente_nombre].ejecutar(tarea)
    
    def obtener_estadisticas(self) -> Dict[str, Any]:
        """Obtiene estadísticas del flujo de trabajo"""
        stats = {
            'agentes_activos': ['investigador', 'analista', 'redactor'],
            'tareas_completadas': len(self.resultados),
            'memoria_investigador': len(self.investigador.obtener_historial()),
            'memoria_analista': len(self.analista.obtener_historial()),
            'memoria_redactor': len(self.redactor.obtener_historial())
        }
        return stats

# PASO 6: Función principal de ejecución
# =====================================

def ejecutar_sistema_multiagente(tema: str):
    """
    Función principal para ejecutar el sistema multiagente
    
    Args:
        tema (str): Tema de investigación
    """
    # Crear coordinador
    coordinador = CoordinadorMultiagente()
    
    # Ejecutar flujo completo
    resultados = coordinador.ejecutar_flujo_completo(tema)
    
    # Mostrar resultados
    print("\n" + "="*60)
    print("RESULTADOS FINALES")
    print("="*60)
    
    for fase, resultado in resultados.items():
        print(f"\n{fase.upper()}:")
        print("-" * 40)
        print(resultado[:500] + "..." if len(resultado) > 500 else resultado)
    
    # Guardar documento final
    if 'documento_final' in resultados:
        with open(f"documento_{tema.replace(' ', '_')}.md", "w", encoding="utf-8") as f:
            f.write(resultados['documento_final'])
        print(f"\n📄 Documento guardado como: documento_{tema.replace(' ', '_')}.md")
    
    return coordinador

# PASO 7: Funciones de utilidad adicionales
# =========================================

async def ejecutar_tareas_paralelas(coordinador: CoordinadorMultiagente, tareas: List[Dict]):
    """
    Ejecuta múltiples tareas en paralelo
    
    Args:
        coordinador: Instancia del coordinador
        tareas: Lista de diccionarios con 'agente' y 'tarea'
    """
    async def ejecutar_tarea(agente_nombre, tarea_desc):
        return coordinador.ejecutar_tarea_personalizada(agente_nombre, tarea_desc)
    
    # Crear tareas asíncronas
    tareas_async = [
        ejecutar_tarea(tarea['agente'], tarea['tarea']) 
        for tarea in tareas
    ]
    
    # Ejecutar en paralelo
    resultados = await asyncio.gather(*tareas_async)
    return resultados

def crear_agente_personalizado(nombre: str, rol: str, prompt: str, herramientas: List[Tool]) -> AgenteBase:
    """
    Crea un agente personalizado
    
    Args:
        nombre: Nombre del agente
        rol: Rol del agente
        prompt: Prompt del sistema
        herramientas: Lista de herramientas
    
    Returns:
        AgenteBase: Nuevo agente personalizado
    """
    return AgenteBase(nombre, rol, herramientas, prompt)

# PASO 8: Ejemplo de uso
# ======================

if __name__ == "__main__":
    # Tema de ejemplo
    tema_ejemplo = "Blockchain en el sector financiero"
    
    # Ejecutar sistema
    coordinador = ejecutar_sistema_multiagente(tema_ejemplo)
    
    # Obtener estadísticas
    stats = coordinador.obtener_estadisticas()
    print(f"\n📊 Estadísticas del sistema: {stats}")
    
    # Ejemplo de tarea personalizada
    print("\n🔧 Ejecutando tarea personalizada...")
    tarea_custom = coordinador.ejecutar_tarea_personalizada(
        'investigador', 
        'Busca las últimas noticias sobre regulaciones de blockchain'
    )
    print(f"Resultado tarea personalizada: {tarea_custom[:200]}...")

# CONFIGURACIONES AVANZADAS
# =========================

# Para usar diferentes modelos:
# from langchain_anthropic import ChatAnthropic
# llm_anthropic = ChatAnthropic(model="claude-3-sonnet-20240229")

# Para agregar callbacks personalizados:
# handler = StdOutCallbackHandler()
# executor = AgentExecutor(..., callbacks=[handler])

# Para memoria persistente:
# from langchain.memory import FileChatMessageHistory
# history = FileChatMessageHistory("chat_history.json")
# memory = ConversationBufferMemory(chat_memory=history)